# Preprocess data for JPLE

## Import packages

In [1]:
import os
import subprocess
from typing import List
import urllib.request

import pandas as pd

## Build the profile HMM file

In [2]:
def extract_hmm(hmm_file: str, hmm_acc_list: List[str]) -> List[str]:
    """
    Extract HMMs from a Pfam-A file.

    Parameters
    ----------
    hmm_file : str
        Input Pfam-A HMM file.
    hmm_acc_list : List[str]
        List of HMM accessions.

    Returns
    -------
    hmm_line_list : List[str]
        List of selected HMM lines.

    """
    with open(hmm_file) as infile:
        write_flag = False
        selected_hmm_line_list = []
        current_hmm_line_list = []
        added_acc_list = []

        # Iterate over the lines of Pfam-A
        for line in infile:

            # Start of HMM
            if line.startswith('HMMER3/f'):
                current_hmm_line_list = [line]
                write_flag = None
                continue
            current_hmm_line_list.append(line)

            # Look for ACC line
            if line.startswith('ACC'):
                current_acc = line.split()[1].split('.')[0]
                if current_acc in hmm_acc_list:
                    added_acc_list.append(current_acc)
                    write_flag = True

            # End of HMM
            if line.strip() == '//':
                if write_flag:
                    selected_hmm_line_list += current_hmm_line_list
                    if len(added_acc_list) == len(hmm_acc_list):
                        return selected_hmm_line_list
                current_hmm_line_list = []
                write_flag = False
    return selected_hmm_line_list


def build_hmm(hmm_acc_path: str, pfam_url: str, rrm_hmm_path: str, output_hmm_path: str) -> None:
    """
    Build the profile HMM file, which consists of those selected by accession and the custom RRM_1 HMM.

    Parameters
    ----------
    hmm_acc_path : str
        Path to the HMM accessions.
    pfam_url : str
        URL of the Pfam-A file.
    rrm_hmm_path : str
        Path to the RRMs_3D_hmm_extended HMM file.
    output_hmm_path : str
        Path to the output HMM file.

    """

    # Read the list of HMM accessions
    with open(hmm_acc_path) as f:
        hmm_acc_list = f.read().splitlines()

    # Download the profile HMMs
    hmm_gz_file = pfam_url.split('/')[-1]
    if not os.path.exists(hmm_gz_file):
        print(f'Downloading {pfam_url}...')
        urllib.request.urlretrieve(pfam_url, hmm_gz_file)
    else:
        print(f'{hmm_gz_file} already exists.')

    # Unpack the profile HMMs
    hmm_file = hmm_gz_file.rstrip('.gz')
    if not os.path.exists(hmm_file):
        print(f'Unpacking {hmm_gz_file}...')
        subprocess.run(['gunzip', '-k', hmm_gz_file], check=True)
    else:
        print(f'{hmm_file} already exists.')

    # Extract HMMs
    selected_hmm_line_list = extract_hmm(hmm_file, hmm_acc_list)
    with open(rrm_hmm_path) as f:
        rrm_hmm_line_list = f.readlines()
    hmm_line_list = selected_hmm_line_list + rrm_hmm_line_list

    # Write HMMs
    with open(output_hmm_path, 'w') as f:
        f.writelines(hmm_line_list)

In [3]:
build_hmm(hmm_acc_path='../data/raw/hmm_acc.txt',
          pfam_url='https://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam38.0/Pfam-A.hmm.gz',
          rrm_hmm_path='../data/raw/RRMs_3D_hmm_extended.hmm',
          output_hmm_path = '../data/processed/domain_rbp.hmm')

Pfam-A.hmm.gz already exists.
Pfam-A.hmm already exists.


## Select the RBPs

In [ ]:
def select_rbp(zscore_eupri_path: str, metadata_eupri_path: str, fasta_train_path: str, zscore_train_path: str) -> None:
    """
    Select the RBPs. One RBP is selected to represent each domain composition.

    Parameters
    ----------
    zscore_eupri_path : str
        Path to the EuPRI k-mer Z-scores.
    metadata_eupri_path : str
        Path to the EuPRI metadata.
    fasta_train_path : str
        Path to the output FASTA file containing data of the selected RBPs.
    zscore_train_path : str
        Path to the output k-mer Z-scores containing data of the selected RBPs.

    """

    # Read the Z-scores
    zscore_df = pd.read_csv(zscore_eupri_path, sep='\t', index_col=0)

    # Compute the mean top-10 Z-score for each protein
    zscore_mean_dict = zscore_df.apply(lambda col: col.nlargest(10).mean()).to_dict()

    # Read the metadata
    metadata_df = pd.read_csv(metadata_eupri_path, sep='\t', index_col=0)

    # Filter for RBPs containing only RRMs or KH domains
    metadata_df = metadata_df[metadata_df['construct_domains'].apply(lambda x: set(x.split(';')).issubset(['RRMs_3D_hmm_extended', 'KH_1']))]
    metadata_df['gene_id'] = metadata_df['gene_id'].replace('', pd.NA)
    metadata_df['gene_id'] = metadata_df['gene_id'].fillna(metadata_df['gene_name'])

    # Select the top sequence for each domain combination
    # It is manually verified that constructs that contain the same (protein_id, construct_domains) refer to the same domains
    group_list_list = metadata_df.groupby(['gene_id', 'construct_domains'])['rnacompete_id'].apply(list).to_list()
    top_list = [max(group_list, key=lambda x: zscore_mean_dict[x]) for group_list in group_list_list]
    tmp_df = metadata_df.set_index('rnacompete_id', drop=False).loc[top_list]

    # Select the top sequence for each unique construct sequence
    group_list_list = tmp_df.groupby('construct_aa_seq')['rnacompete_id'].apply(list).to_list()
    top_list = sorted([max(group_list, key=lambda x: zscore_mean_dict[x]) for group_list in group_list_list])

    # Write the selected protein sequences to a FASTA file
    seq_list = metadata_df.set_index('rnacompete_id').loc[top_list]['construct_aa_seq'].to_list()
    with open(fasta_train_path, 'w') as f:
        for rnacompete_id, seq in zip(top_list, seq_list):
            f.write(f'>{rnacompete_id}\n')
            f.write(f'{seq}\n')

    # Subset the Z-scores of the selected proteins
    zscore_df.loc[:, top_list].to_csv(zscore_train_path, sep='\t')

In [ ]:
select_rbp(zscore_eupri_path = '../data/raw/zscore_eupri.tsv',
           metadata_eupri_path = '../data/raw/rnacompete_metadata_eupri.tsv',
           fasta_train_path = '../data/processed/seq_train.fasta',
           zscore_train_path = '../data/processed/zscore_train.tsv')